In [19]:
## setup 

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


#author : steeve laquitaine modified from everett wetchler
#purpose Helpers functions to check dataset
#Helper functions

def percentify_axis(ax, which):
    which = which.lower()
    if which in ('x', 'both'):
        ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])
    if which in ('y', 'both'):
        ax.set_yticklabels(['%.0f%%' % (t*100) for t in ax.get_yticks()])

color_idx = 0
CYCLE_COLORS = sns.color_palette()
def next_color():
    global color_idx
    c = CYCLE_COLORS[color_idx] 
    color_idx = (color_idx + 1) % len(CYCLE_COLORS)
    return c

def count_unique(s):
    values = s.unique()
    return sum(1 for v in values if pd.notnull(v))

def missing_pct(s,N):
    missing = N - s.count()
    return missing * 100.0 / N

def complete_pct(s,N):
    return 100 - missing_pct(s,N)

def summarize_completeness_uniqueness(df,N):
    print '*** How complete is each feature? How many different values does it have? ***'
    rows = []
    for col in df.columns:
        rows.append([col, '%.0f%%' % complete_pct(df[col],N), count_unique(df[col])])
    dframe = pd.DataFrame(rows, columns=['Column Name', 'Complete (%)','Unique Values'])
    pd.set_option('display.max_colwidth',999,'display.max_row',999)
    return dframe

def summarize_completeness_over_time(df, time_col, transpose=True):
    print '*** Data completeness over time per column ***'
    x = df.groupby(time_col).count()
    x = x.div(df.groupby(time_col).size(), axis=0)
    for col in x.columns:
        x[col] = x[col].apply(lambda value: '%.0f%%' % (value * 100))
    if transpose:
        return x.T
    pd.set_option('display.max_colwidth',999,'display.max_row',999)
    return x

def plot_top_hist(df, col, top_n=10, skip_below=.01):
    '''Plot a histogram of a categorical dataframe column, limiting to the most popular.'''
    counts = df[col].value_counts(True, ascending=True)
    if counts.max() < skip_below:
        print 'Skipping "%s" histogram -- most common value is < %.0f%% of all cases' % (col, skip_below*100)
        return
    fig, ax = plt.subplots(1)
    explanation = ''
    if len(counts) > top_n:
        explanation = ' (top %d of %d)' % (top_n, len(counts))
        counts = counts.iloc[-top_n:]
    explanation += ' -- %.0f%% missing' % (missing_pct(df[col]))
    counts.plot(kind='barh', ax=ax, color=next_color())
    ax.set_title('Rows by "%s"%s' % (col, explanation))
    ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])
    
print "Complete"

Complete


In [2]:
## import data 
infile = "/Users/alysonkane/Desktop/HHS/health-insurance-marketplace/BenefitsCostSharing.csv"

df = pd.read_csv(infile)
N = len(df)
print 'Read %d rows %d cols\n' % df.shape 
df.head(5)


/Users/alysonkane/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,6,9,10,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,BenefitName,BusinessYear,CoinsInnTier1,CoinsInnTier2,CoinsOutofNet,CopayInnTier1,CopayInnTier2,CopayOutofNet,EHBVarReason,Exclusions,...,LimitUnit,MinimumStay,PlanId,QuantLimitOnSvc,RowNumber,SourceName,StandardComponentId,StateCode,StateCode2,VersionNum
0,Routine Dental Services (Adult),2014,20%,NaN,20%,No Charge,NaN,No Charge,Above EHB,NaN,...,Dollars per Year,NaN,21989AK0010001-00,Yes,68,HIOS,21989AK0010001,AK,AK,6
1,Dental Check-Up for Children,2014,20%,NaN,20%,No Charge,NaN,No Charge,Substantially Equal,NaN,...,Visit(s) per 6 Months,NaN,21989AK0010001-00,Yes,104,HIOS,21989AK0010001,AK,AK,6
2,Basic Dental Care - Child,2014,40%,NaN,40%,No Charge,NaN,No Charge,Substantially Equal,NaN,...,NaN,NaN,21989AK0010001-00,NaN,110,HIOS,21989AK0010001,AK,AK,6
3,Orthodontia - Child,2014,50%,NaN,50%,No Charge,NaN,No Charge,Additional EHB Benefit,NaN,...,NaN,NaN,21989AK0010001-00,NaN,111,HIOS,21989AK0010001,AK,AK,6
4,Major Dental Care - Child,2014,50%,NaN,50%,No Charge,NaN,No Charge,Substantially Equal,NaN,...,NaN,NaN,21989AK0010001-00,NaN,112,HIOS,21989AK0010001,AK,AK,6


In [7]:
df_sub = df[0:5000]
print 'Read %d rows %d cols\n' % df_sub.shape 

Read 5000 rows 32 cols



In [15]:
## remove duplicated fields 
df_cleaned = df
print sum(df_cleaned['StateCode']==df_cleaned['StateCode2'])*100/N,'% of the rows match between "StateCode" and "StateCode2" so "StateCode2" was removed.'
df_cleaned = df_cleaned.drop('StateCode2',1)
print sum(df_cleaned['IssuerId']==df_cleaned['IssuerId2'])*100/N,'% of the rows match between "IssuerId" and "IssuerId2" so "IssuerId2" was removed.'
df_cleaned = df_cleaned.drop('IssuerId2',1)
print 'Read %d rows %d cols\n' % df_cleaned.shape 

100 % of the rows match between "StateCode" and "StateCode2" so "StateCode2" was removed.
100 % of the rows match between "IssuerId" and "IssuerId2" so "IssuerId2" was removed.
Read 5048408 rows 30 cols



In [49]:
## summarize data 
summarize_completeness_uniqueness(df_cleaned,N)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,BenefitName,100%,861
1,BusinessYear,100%,3
2,CoinsInnTier1,78%,115
3,CoinsInnTier2,9%,37
4,CoinsOutofNet,78%,60
5,CopayInnTier1,78%,516
6,CopayInnTier2,9%,278
7,CopayOutofNet,78%,232
8,EHBVarReason,40%,12
9,Exclusions,9%,1720


In [ ]:
## variables with expected values
VarWithExpecVals = ['BusinessYear','StateCode','IssuerId','SourceName','IsEHB','IsStateMandate', 'IsCovered', 
                    'QuantLimitOnSvc', 'LimitUnit', 'EHBVarReason', 'IsSubjToDedTier1', 'IsSubjToDedTier2',
                    'IsExclFromInnMOOP', 'IsExclFromOonMOOP']

## expected values
expectedVals = {}
expectedVals["BusinessYear"] = [2014,2015,2016]
expectedVals["StateCode"] = ['AK','AL','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
expectedVals["IssuerId"] = np.arange(99999)
expectedVals["SourceName"] = ['HIOS','SERFF','OPM']
expectedVals["IsEHB"] = ['Yes','']
expectedVals["IsStateMandate"] = ['Yes','']
expectedVals["IsCovered"] = ['Covered','Not Covered', '']
expectedVals["QuantLimitOnSvc"] = ['Yes', 'No', '']
expectedVals["LimitUnit"] = ['Hours per week', 'Hours per month', 'Hours per year', 'Days per week', 'Days per month',
                             'Days per year', 'Months per year', 'Visits per week', 'Visits per month', 'Visits per year',
                             'Lifetime visits', 'Treatments per week', 'Treatments per month', 'Lifetime treatments',
                             'Lifetime admissions', 'Procedures per week', 'Procedures per month', 'Procedures per year',
                             'Lifetime procedures', 'Dollar per year', 'Dollar per visit', 'Days per admission',
                             'Procedures per episode','']
expectedVals["EHBVarReason"] = ['Above EHB', 'Substituted', 'Substantially Equal', 'Using Alternate Benchmark ', 
                                'Other Law/Regulation ', 'Additional EHB Benefit ', 'Dental Only Plan Available']
expectedVals["IsSubjToDedTier1"] = ['Yes','No'] 
expectedVals["IsSubjToDedTier2"] = ['Yes','No'] 
expectedVals["IsExclFromInnMOOP"] = ['Yes','No']   
expectedVals["IsExclFromOonMOOP"] = ['Yes','No']      
    
print '*** Are there any values outside their expected range ? ***'
rows = []

#Check validity
for col in VarWithExpecVals:    
    #check among existing values    
    s = pd.Series(list(df[col]))            
    s_exist = s[pd.notnull(s)]
    numValid = np.sum(s_exist.isin(expectedVals[col]))    
    percValid = numValid*100/len(s_exist)
    rows.append([col,'%.0f%%' % percValid,numValid])
tab = pd.DataFrame(rows,columns=['Column Name', 'Valid value(%)','Unique Valid Values'])

#fill up cleaned dataset
for col in VarWithExpecVals:    
    df_cleaned[col] = df[col]
    
tab    

In [57]:
### Update Null Values to "No" When Specified in Data Dictionary 

#loop through variables
TheseVars = ['IsEHB','IsStateMandate','QuantLimitOnSvc']
for col in TheseVars:  
     df_cleaned.loc[pd.isnull(df[col]) == True, col] = 'No'

### Update blanks with "Not Covered" as specified in data dictionary 
df_cleaned.loc[pd.isnull(df['IsCovered']) == True, col] = 'Not Covered'

#loop through variables
TheseVars = ['IsSubjToDedTier1','IsSubjToDedTier2','IsExclFromInnMOOP', 'IsExclFromOonMOOP','QuantLimitOnSvc']
for col in TheseVars:  
    df_cleaned[col] = df[col].str.lower()

In [64]:
## Check LimitQty and LimitUnit when QuantLimitOnSvc = YES
df_sub = df_cleaned[df_cleaned['QuantLimitOnSvc'] == 'yes']
n = len(df)

print(missing_pct(df_sub['LimitQty'],n))
print(missing_pct(df_sub['LimitUnit'],n))


86.5880491434
86.5880491434


In [66]:
summarize_completeness_over_time(df_cleaned,'ImportDate')

*** Data completeness over time per column ***


ImportDate,1/16/2015 17:32,1/16/2015 20:59,1/17/2015 3:15,1/20/2015 13:44,1/21/2015 12:15,1/21/2015 9:35,1/22/2015 10:35,1/22/2015 16:02,1/23/2015 12:43,1/23/2015 9:32,...,9/29/2014 21:43,9/3/2014 4:28,9/4/2014 3:25,9/4/2014 6:14,9/5/2014 3:32,9/5/2014 6:40,9/6/2014 3:39,9/7/2014 12:14,9/8/2014 9:42,9/9/2014 16:12
BenefitName,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%,...,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%
BusinessYear,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%,...,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%
CoinsInnTier1,78%,78%,76%,75%,78%,79%,79%,76%,79%,71%,...,81%,73%,80%,76%,68%,76%,74%,72%,69%,76%
CoinsInnTier2,6%,0%,32%,0%,12%,6%,0%,18%,22%,0%,...,0%,0%,0%,0%,0%,1%,2%,1%,0%,1%
CoinsOutofNet,78%,78%,76%,75%,78%,79%,79%,76%,79%,71%,...,81%,73%,80%,76%,68%,76%,74%,72%,69%,76%
CopayInnTier1,78%,78%,76%,75%,78%,79%,79%,76%,79%,71%,...,81%,73%,80%,76%,68%,76%,74%,72%,69%,76%
CopayInnTier2,6%,0%,32%,0%,12%,6%,0%,18%,22%,0%,...,0%,0%,0%,0%,0%,1%,2%,1%,0%,1%
CopayOutofNet,78%,78%,76%,75%,78%,79%,79%,76%,79%,71%,...,81%,73%,80%,76%,68%,76%,74%,72%,69%,76%
EHBVarReason,47%,35%,41%,38%,47%,39%,35%,65%,38%,58%,...,43%,71%,56%,29%,50%,26%,33%,45%,49%,42%
Exclusions,6%,30%,29%,0%,11%,0%,7%,21%,2%,18%,...,2%,0%,37%,3%,9%,3%,19%,12%,20%,23%


In [67]:
print 'Read %d rows %d cols\n' % df_cleaned.shape 

Read 5048408 rows 30 cols



In [68]:
df_cleaned.to_csv('/Users/alysonkane/Desktop/HHS/health-insurance-marketplace/BenefitsCostSharing_clean.csv')